In [ ]:
from exp.run import run_experiment
from exp.mappings import alg_map
from exp.train import train_model
from exp.features import load_cv_results
import pandas as pd
import json
import numpy as np
import os
import pickle
import inspect
import kaggle_files

### Load results from CSV File and re-produce models ranked by CV scores

In [ ]:
save_results= "exp9.csv"
score_df = load_cv_results(save_results)
score_df = score_df.sort_values(by="cv_score_n_folds_5_shuffle_False_rs_None", axis=0)

In [ ]:
display(score_df)

### Load best model from CSV File

In [ ]:
i = 7
k = 0
n_folds = 5
shuffle = False

In [ ]:
# retrieve top scoring row
best = score_df.iloc[i]
display(best)

# retrieve model parameters from pandas row
alg = best["alg"]
params_json = best["params_json"]
fs = best["feature_set"]

print("alg: {}".format(alg))
print("params_json: {}".format(params_json))
print("feature_set: {}".format(fs))

# retrieve relevant values
alg_cls = alg_map[alg]
params = json.loads(params_json)
params.pop("bagging_seed", None)
params["random_state"] = k

# train algorithm
score, mad, oof, preds = train_model(params=params, fs=fs, n_fold=n_folds, shuffle=shuffle, alg=alg, test_eval=True)

In [ ]:
files_dir = os.path.dirname(inspect.getfile(kaggle_files))
submission_dir = os.path.join(files_dir, "submission")
sample_submission_file = os.path.join(submission_dir, "sample_submission.csv")
submission = pd.read_csv(sample_submission_file, index_col='seg_id')
submission['time_to_failure'] = preds
print(submission.head())
submission.to_csv('submission_n_folds_'+str(n_folds)+"_shuffle_"+str(shuffle)+"_"+str(i)+'_seed_'+str(k)+'.csv')